# Load my module

In [104]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
#
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# classifiers
your_module = drive.CreateFile({'id':'1nqIi2cf1R1nN5Y3XwQ-Pm_TFsvQhKbh0'})
your_module.GetContentFile('PU_Bayesian_classfiers.py')
# PU generator
your_module1 = drive.CreateFile({'id':'1kXxxRqnEAvEVoW2kb38lOL6pS3-CZ7bo'})
your_module1.GetContentFile('PU_generator.py')
# get_cv
your_module2 = drive.CreateFile({'id':'1M0dIks11EF6Bc-TKVCAfW0VQPKFhlxJ5'})
your_module2.GetContentFile('get_cv.py')
#
from PU_Bayesian_classfiers import *
from PU_generator import *
from get_cv import *

In [0]:
def get_cv(X, y, case_control = True, nl = None, nu = None, c = None,n_T = None, random_state = 42, pos_label = '1', cls = None, M = None, runs=10, verbose = True):  
  """ Get CLL, accuracy, precision, recall under multiple runs
      Parameters
      ----------
      cls:  ___{PNB, PTAN,PSTAN, WNB, WTAN}___
            Model class
      X : {array-like, sparse matrix}, shape (n_samples, n_features)
            Feature matrix.
      Y : {array-like, sparse matrix}, shape (n_samples,)
            Target.
      M : {array-like, sparse matrix}, shape (n_features, n_features)
            Contact matrix
      nl : Scalar
            Number of labeled examples
      nu : Scalar
            Number of Unlabeled examples
      runs : Scalar
            Number of runs, all metrics are average of all runs.
      random_state : Scalar
            Random state on PU generator, make results reproducible
      Pos_label : str type
            Positive label in data. Default is '1'
      case_control : bool

      Returns
      -------
      CLL, accuracy, precision, recall of all runs.
      
  """
  X,y = check_X_y(X,y)
  Accuracy = []
  Precision = []
  Recall = []
  CLL = []
  for i in range(runs):
    # generate PU data, with different seed i*random_state
    pu_object = PUgenerator()
    pu_object.fit(X,y,case_control,nl,nu,c,n_T,random_state*(i+1),pos_label) # generate case control or single training data
    model = cls()
    model.fit(pu_object.X_1abeled_,pu_object.X_Unlabeled_, pu_object.prevalence_,M, case_control)
    Accuracy.append(accuracy_score(pu_object.y_Unlabeled_,model.predict(pu_object.X_Unlabeled_)) )
    CLL.append(model.Conditional_log_likelihood(pu_object.y_Unlabeled_,model.predict_proba(pu_object.X_Unlabeled_)) )
    Precision.append(precision_score(pu_object.y_Unlabeled_,model.predict(pu_object.X_Unlabeled_), 
         average='macro') )
    Recall.append(recall_score(pu_object.y_Unlabeled_,model.predict(pu_object.X_Unlabeled_), 
         average='macro' ) )
    if verbose:
        print("accuracy in %s -th run is %s" % (i+1,Accuracy[i]) )
        print("CLL in %s -th run is %s" % (i+1,CLL[i]))
        print("Precision in %s -th run is %s" % (i+1,Precision[i]) )
        print("Recall in %s -th run is %s" % (i+1,Recall[i]) )
        #if not case_control:
        #  print("estimated p(s=1|y=1) in %s -th run = %s" % (i+1, model.c_ ))
        print(10*'__')
  return np.array(Accuracy), np.array(CLL), np.array(Precision),np.array(Recall)

## read data 

In [0]:
# P450
def readData(filename):
	fr = open(filename)
	returnData = []
	headerLine=fr.readline()###move cursor 
	for line in fr.readlines():
		lineStrip = line.strip().replace('"','')
		lineList =	lineStrip.split('\t')
		returnData.append(lineList)###['3','2',...]
	return returnData

"""first case P450 = [['1','1',....],[],[].....,[]]  second case P450 = array([['1','1',....],[],[].....,[]]), third case P450 = """
P450 = readData('/content/drive/My Drive/Protein project/P450.txt') ### [[],[],[],....[]]
P450 = np.array(P450) ### either [['1','1',....],[],[].....,[]] or array([['1','1',....],[],[].....,[]]) works, but note that keys are '1', '0' 
#P450 = P450.astype(int) ### for shap array [[1,1,....],[],[].....,[]], keys are 1, 0 

M=np.matrix([[245,   9,   0,   3,   0,   2,  65,   8],
[9, 218,  17,  17,  49,  10,  50,  17],
[0,  17, 175,  16,  25,  13,   0,  46],
[3,  17,  16, 194,  19,   0,   0,   3],
[0,  49,  25,  19, 199,  10,   0,   3],
[2,  10,  13,   0,  10, 249,  50,  74],
[65,  50,   0,   0,   0,  50, 262,  11],
[8,  17,  46,   3,   3,  74,  11, 175]])

X = P450[:,0:8]
y = P450[:,-1]

In [0]:
# lactamse
def readData2(filename):
	fr = open(filename)
	returnData = []
	headerLine=fr.readline()###move cursor 
	for line in fr.readlines():
		linestr = line.strip().replace(', ','')
		lineList =	list(linestr)
		returnData.append(lineList)###['3','2',...]
	return returnData

lactamase = readData2('/content/drive/My Drive/Protein project/lactamase.txt')
lactamase = np.array(lactamase)
#lactamase = lactamase.astype(int)
M2 = np.matrix([[101, 5, 0, 2, 0, 14, 4, 37],
[5 ,15, 14 ,1 ,7 ,7, 0 ,19],
[0, 14, 266, 15, 14, 2, 26, 4],
[2, 1, 15, 28, 2 ,15, 4, 0],
[0, 7, 14, 2, 32, 9 ,0, 8],
[14, 7, 2 ,15, 9, 29, 7, 9],
[4, 0, 26, 4 ,0 ,7 ,72, 21],
[37, 19, 4, 0, 8, 9, 21, 211]])

X2 = lactamase[:,0:8]
y2 = lactamase[:,-1]

In [0]:
# lymph
def readarff(filename):
  arrfFile = open(filename)
  lines = [line.rstrip('\n') for line in arrfFile]
  data = [[]]
  index = 0
  for line in lines :
      if(line.startswith('@attribute')) :
          index+=1
      elif(not line.startswith('@data') and not line.startswith('@relation') and not line.startswith('%')) :
          data.append(line.split(','))
  del data[0]
  return data


lymph_train = readarff("/content/drive/My Drive/Protein project/github_bn_code/lymph_train.arff.txt"); lymph_train = np.array(lymph_train)  
lymph_test = readarff("/content/drive/My Drive/Protein project/github_bn_code/lymph_test.arff.txt") ;lymph_test = np.array(lymph_test)

lymph = np.concatenate((lymph_train,lymph_test))

X10 = lymph[:,0:18]
y10 = lymph[:,-1]

In [0]:
# vote
vote_train = readarff("/content/drive/My Drive/Protein project/github_bn_code/vote_train.arff.txt") ;vote_train = np.array(vote_train)
vote_test = readarff("/content/drive/My Drive/Protein project/github_bn_code/vote_test.arff.txt")  ; vote_test = np.array(vote_test)

vote = np.concatenate((vote_train,vote_test))
X11 = vote[:,0:16]
y11 = vote[:,-1]

# Simulation

## Vote

### case-control

#### nl = 400, nu = 450

In [294]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X11,y11,case_control = True,nl = 400,nu = 450, cls = PNB, M =None, verbose = False,runs= 20, pos_label= "'democrat'")
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9022222222222223 +/- 0.013014711523863395
-379.90030157585136 +/- 69.07672661220352
0.8939573160407107 +/- 0.014135237180532926
0.9056259739573965 +/- 0.013830652873394819


In [295]:
# PTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X11,y11,case_control = True,nl = 400,nu = 450, cls = PTAN, M =None, verbose = False,runs= 20, pos_label= "'democrat'")
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9123333333333334 +/- 0.028235998387488834
-217.5432327647492 +/- 66.48338871347454
0.9132965456353311 +/- 0.02972407418106014
0.9006698562087854 +/- 0.03189976720981765


In [296]:
# WNB case control
Accuracy, CLL, Precision,Recall = get_cv(X11,y11,case_control = True,nl = 400,nu = 450, cls = WNB, M =None, verbose = False,runs= 20, pos_label= "'democrat'")
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8732222222222221 +/- 0.014409787483118622
-357.02037983411725 +/- 57.89022247886137
0.8717452748560051 +/- 0.014189731145388436
0.8930529825108209 +/- 0.011893450730670104


#### nl = 200, nu = 450

In [297]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X11,y11,case_control = True,nl = 200,nu = 450, cls = PNB, M =None, verbose = False,runs= 20, pos_label= "'democrat'")
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9047777777777778 +/- 0.013319900641011554
-367.63895107967426 +/- 75.76803679000265
0.8965837962325514 +/- 0.01428473966433592
0.9080959890157706 +/- 0.013995102070493969


In [298]:
# PTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X11,y11,case_control = True,nl = 200,nu = 450, cls = PTAN, M =None, verbose = False,runs= 20, pos_label= "'democrat'")
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9022222222222223 +/- 0.02511209437602959
-267.3677856296165 +/- 77.1253088017882
0.9047369018791999 +/- 0.027441938361262873
0.8883508191876356 +/- 0.03100872246191902


In [299]:
# WNB case control
Accuracy, CLL, Precision,Recall = get_cv(X11,y11,case_control = True,nl = 200,nu = 450, cls = WNB, M =None, verbose = False,runs= 20, pos_label= "'democrat'")
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8724444444444444 +/- 0.014459820875553284
-353.4640855940881 +/- 64.69382541998264
0.8710208186227305 +/- 0.014195997877639203
0.8922469566362417 +/- 0.011599214788490906


### single training

## Lactamase

### single training

#### c = 0.7, nt = 550

In [254]:
# PNB single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PNB, M =M2,runs= 20,c = 0.7,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9166051599615039 +/- 0.013660949823274423
-161.38222440222216 +/- 8.784088206025702
0.6702903010011958 +/- 0.043070133518396427
0.6461164994895556 +/- 0.04557557747380148


In [255]:
# PTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PTAN, M =M2,runs= 20,c = 0.7,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9435343282431233 +/- 0.01079264605741632
-109.10246417239884 +/- 10.855703899151903
0.7849634951206227 +/- 0.053699657329306386
0.7887229444598154 +/- 0.05405608703864189


In [256]:
# PSTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PSTAN, M =M2,runs= 20,c = 0.7,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9387782865325602 +/- 0.012815987479028755
-113.981599487679 +/- 14.012645279025598
0.76745302952525 +/- 0.052717745641758754
0.7997622954869843 +/- 0.043744909026525176


In [257]:
# PESTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PESTAN, M =M2,runs= 20,c = 0.7,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9302560761115826 +/- 0.012396895082791156
-117.43055394538223 +/- 12.571726735420407
0.7398385497054089 +/- 0.04456932664741783
0.8130768399794068 +/- 0.043798481975348545


In [258]:
# WNB single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = WNB, M =M2,runs= 20,c = 0.7,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9136668261593389 +/- 0.014343756891360035
-162.69910501895146 +/- 10.37589517013352
0.6631431010271285 +/- 0.04444454478219113
0.6422359884524217 +/- 0.03449032680254685


In [261]:
# WTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = WTAN, M =M2,runs= 20,c = 0.7,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9381376944571027 +/- 0.01125175811362483
-121.6151570989095 +/- 13.889510543510731
0.7636854731100482 +/- 0.04635691886931679
0.7840231833134036 +/- 0.044929037799215074


#### c = 0.5, nt = 550

In [262]:
# PNB single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PNB, M =M2,runs= 20,c = 0.5,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8865769558103807 +/- 0.016864440764258262
-200.69338898275632 +/- 14.064873394848531
0.7021296206527168 +/- 0.04734222796004115
0.6336823734034231 +/- 0.034060206679925015


In [263]:
# PTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PTAN, M =M2,runs= 20,c = 0.5,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9162378188443752 +/- 0.01245540634957535
-148.5354544527907 +/- 14.876703012294458
0.7947604616384748 +/- 0.04864842120593945
0.7573099418347794 +/- 0.03029070766366388


In [264]:
# PSTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PSTAN, M =M2,runs= 20,c = 0.5,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.918822403795647 +/- 0.013982986920591123
-151.07646318351095 +/- 15.581271870962865
0.8014856391851224 +/- 0.04753610077831585
0.7818935167547653 +/- 0.029326542714235126


In [265]:
# PESTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PESTAN, M =M2,runs= 20,c = 0.5,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.91500463806953 +/- 0.014063983554209857
-150.28941061421875 +/- 14.627980386240138
0.7859828850973696 +/- 0.04093642675603927
0.7930313209364028 +/- 0.032543349127421774


In [266]:
# WNB single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = WNB, M =M2,runs= 20,c = 0.5,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8850607001144996 +/- 0.01744539699013794
-200.23889385374534 +/- 15.26507329395255
0.6960471725758387 +/- 0.05117480648951332
0.6196660848473196 +/- 0.03048843910626979


In [267]:
# WTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = WTAN, M =M2,runs= 20,c = 0.5,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9195015637049646 +/- 0.015627152189915332
-159.69762986285724 +/- 16.1126594664372
0.8113626651040009 +/- 0.05111960136166534
0.7493824091491157 +/- 0.03403481822736961


#### c = 0.3, nt = 550

In [268]:
# PNB single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PNB, M =M2,runs= 20,c = 0.3,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8568369389406879 +/- 0.019261750630748236
-247.44123232772964 +/- 18.803518638313605
0.7086019149169778 +/- 0.048690364690298096
0.6325411022167007 +/- 0.040185958917696815


In [270]:
# PTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PTAN, M =M2,runs= 20,c = 0.3,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.888578760485341 +/- 0.016821425847962212
-197.57837660588933 +/- 21.418851057743613
0.7921414081503648 +/- 0.05213161021417844
0.7295602080945321 +/- 0.03604780905878835


In [271]:
# PSTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PSTAN, M =M2,runs= 20,c = 0.3,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8860354621702834 +/- 0.01782974125430805
-202.15272043412813 +/- 21.986510436300637
0.7831725314607055 +/- 0.04610408410548159
0.7323120741052439 +/- 0.03281823592673694


In [272]:
# PESTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PESTAN, M =M2,runs= 20,c = 0.3,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8933801459704569 +/- 0.015728731865223618
-195.44617330564398 +/- 13.783167667260138
0.7960737280468576 +/- 0.040706661358293594
0.7581856830609595 +/- 0.03662117920809118


In [273]:
# WNB single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = WNB, M =M2,runs= 20,c = 0.3,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8602141168016703 +/- 0.018052641346153846
-244.48312063365773 +/- 16.34590012353444
0.7243423066812609 +/- 0.04967035600140688
0.6157572646642381 +/- 0.028088870287868974


In [274]:
# WTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = WTAN, M =M2,runs= 20,c = 0.3,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8837863219495548 +/- 0.021323359176385587
-208.85499673507144 +/- 15.725691753129526
0.7971612258706573 +/- 0.057532659031500705
0.6827425622300478 +/- 0.04268275738355029


#### c = 0.2, nt = 550

In [275]:
# PNB single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PNB, M =M2,runs= 20,c = 0.2,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.840148809867879 +/- 0.020814337325752292
-282.59597883513703 +/- 31.416395157360036
0.7029643863231418 +/- 0.05490461184585613
0.6213113870585877 +/- 0.03776288516359428


In [276]:
# PTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PTAN, M =M2,runs= 20,c = 0.2,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8719520153011278 +/- 0.017724657405024286
-239.4920513625541 +/- 27.01243149421369
0.782863536155834 +/- 0.05008021736278948
0.7064706900657687 +/- 0.03851505993486179


In [277]:
# PSTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PSTAN, M =M2,runs= 20,c = 0.2,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8657024021312925 +/- 0.020115609228250765
-243.43037352123707 +/- 27.92525228594986
0.7677990955106917 +/- 0.053207284617753654
0.7056109212611679 +/- 0.03344290066864103


In [278]:
# PESTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PESTAN, M =M2,runs= 20,c = 0.2,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8698762365754004 +/- 0.020661658123442894
-231.90960831840312 +/- 25.623358412190598
0.7746513537270975 +/- 0.05307107417508285
0.7161240969994904 +/- 0.04004980943753634


In [279]:
# WNB single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = WNB, M =M2,runs= 20,c = 0.2,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.844208322908638 +/- 0.019847801110892998
-275.60569113653185 +/- 23.721276677996688
0.7231082760234739 +/- 0.07172827724339466
0.6072495845721315 +/- 0.031000247051951507


In [280]:
# WTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = WTAN, M =M2,runs= 20,c = 0.2,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8619027248774846 +/- 0.022983269425557926
-243.9089567707644 +/- 22.534288440040527
0.7763293098278132 +/- 0.07051256199853345
0.6487842581160098 +/- 0.04168754941808483


#### c = 0.1, nt = 550

In [281]:
# PNB single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PNB, M =M2,runs= 20,c = 0.1,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8105072172963294 +/- 0.019839187020186298
-351.3238247157095 +/- 50.72315230083146
0.6598521554372058 +/- 0.060113063182119834
0.5991284679283149 +/- 0.03462290907922675


In [282]:
# PTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PTAN, M =M2,runs= 20,c = 0.1,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8285760467502291 +/- 0.015387737686156939
-332.5748685453716 +/- 44.95173194721381
0.7050393678837887 +/- 0.05762554440122195
0.6266619153932264 +/- 0.03881902342833181


In [283]:
# PSTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PSTAN, M =M2,runs= 20,c = 0.1,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8206094362846083 +/- 0.017848761041844616
-330.17249116798 +/- 41.069893490535655
0.6867587987294264 +/- 0.052531119493557835
0.6197042135636126 +/- 0.03539772889462277


In [284]:
# PESTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = PESTAN, M =M2,runs= 20,c = 0.1,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8266317581971606 +/- 0.019106050682588034
-311.83689078605687 +/- 38.762879861259684
0.7022775689580747 +/- 0.054834437124974274
0.6343509043328891 +/- 0.040708705632273044


In [285]:
# WNB single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = WNB, M =M2,runs= 20,c = 0.1,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8236569540426354 +/- 0.019911877888848607
-333.86512837738206 +/- 37.19702025231433
0.6959610872721506 +/- 0.07605193194052849
0.5918084266021724 +/- 0.03594946707668431


In [286]:
# WTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = False, cls = WTAN, M =M2,runs= 20,c = 0.1,n_T =550, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8332969188287784 +/- 0.017768945499104707
-309.86367868755383 +/- 34.580714554896076
0.726688419867864 +/- 0.07025255142767564
0.6129876623525085 +/- 0.0338260835053752


### case-control

#### nl = 500, nu = 550

In [233]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 500,nu = 550, cls = PNB, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8521818181818182 +/- 0.016303931483485253
-269.27578073861997 +/- 15.924230567541908
0.8018625212059309 +/- 0.031002835787392782
0.6800829537040748 +/- 0.0309046271314328


In [234]:
# PTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 500,nu = 550, cls = PTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9288181818181818 +/- 0.009711116574522574
-164.2317037318823 +/- 10.333156518743582
0.9077626438416848 +/- 0.020878269505661688
0.8590136644474635 +/- 0.020789223488088593


In [235]:
# PSTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 500,nu = 550, cls = PSTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9215454545454547 +/- 0.007546549756222397
-169.13331398733436 +/- 9.21601116545802
0.8878769764051544 +/- 0.01308955397690897
0.8575557076360847 +/- 0.016400075571617722


In [236]:
# PESTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 500,nu = 550, cls = PESTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9060000000000002 +/- 0.010788729963015838
-175.3798829374728 +/- 12.825181689207087
0.8540574090736698 +/- 0.021332259849477748
0.8479273372946796 +/- 0.020523489404546267


In [237]:
# WNB case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 500,nu = 550, cls = WNB, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8325454545454546 +/- 0.02022926445760863
-275.31120322918184 +/- 16.057009394757422
0.7828059287403406 +/- 0.049870833444245026
0.6131679498503796 +/- 0.02942259212140436


In [238]:
# WTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 500,nu = 550, cls = WTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.902818181818182 +/- 0.015443578996832472
-186.1105615279794 +/- 10.766331119327315
0.912564514323081 +/- 0.02004659218144473
0.7723250195181262 +/- 0.026192826921604427


#### nl = 400, nu = 550

In [218]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 400,nu = 550, cls = PNB, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8503636363636364 +/- 0.01609989476891846
-269.43299141868397 +/- 16.069976122231935
0.7947330808478814 +/- 0.032006895962813
0.6790723462851471 +/- 0.032877445763011784


In [216]:
# PTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 400,nu = 550, cls = PTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9282727272727271 +/- 0.010163538786326188
-165.7169573500551 +/- 10.305723785856465
0.9083051455385709 +/- 0.018897250117092382
0.8572995121577328 +/- 0.020785109522190072


In [221]:
# PSTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 400,nu = 550, cls = PSTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9211818181818183 +/- 0.009258009710604797
-170.43130860694413 +/- 10.13458105511012
0.8886840701780404 +/- 0.015254985840714094
0.8556841027867973 +/- 0.01869373195487822


In [222]:
# PESTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 400,nu = 550, cls = PESTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9068181818181816 +/- 0.010170041891069853
-176.65514255173218 +/- 13.112558690370452
0.8551883271403231 +/- 0.016773403464166628
0.8505739191569852 +/- 0.022473561846262046


In [223]:
# WNB case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 400,nu = 550, cls = WNB, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8349090909090912 +/- 0.020430086386309446
-275.2552843523993 +/- 16.142657960101868
0.7909522334785861 +/- 0.054092275870385384
0.618005360377163 +/- 0.029330951996413647


In [224]:
# WTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 400,nu = 550, cls = WTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.901818181818182 +/- 0.014703684813931434
-187.92333832556935 +/- 10.93572121888214
0.9087416307752821 +/- 0.020331923705900826
0.7717079307687469 +/- 0.02556216633801758


#### nl = 300, nu = 550

In [225]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 300,nu = 550, cls = PNB, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8469090909090908 +/- 0.017090909090909094
-271.1110608994128 +/- 16.600783235391255
0.7863666755858008 +/- 0.038668131523532656
0.6720476613166697 +/- 0.0340208402954


In [226]:
# PTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 300,nu = 550, cls = PTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.925090909090909 +/- 0.010719565798533576
-168.4049149763575 +/- 11.017148579921301
0.9025446012856897 +/- 0.020002754646276186
0.8523069606437673 +/- 0.022489581671697345


In [229]:
# PSTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 300,nu = 550, cls = PSTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9182727272727274 +/- 0.009135798263827597
-174.10293404227204 +/- 10.632714841523683
0.8833490906001409 +/- 0.014930038363885175
0.8512533710853901 +/- 0.019620614837161935


In [230]:
# PESTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 300,nu = 550, cls = PESTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9062727272727272 +/- 0.0111410936151775
-179.45541867578103 +/- 12.985809271024491
0.8549703275793629 +/- 0.015981757592356982
0.8472600477649201 +/- 0.02626995556292458


In [231]:
# WNB case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 300,nu = 550, cls = WNB, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8320909090909092 +/- 0.02151513017497875
-276.23384221128435 +/- 16.313144498423547
0.7793772781060624 +/- 0.056372030736004344
0.6138333527891422 +/- 0.029090725166252523


In [232]:
# WTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 300,nu = 550, cls = WTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9026363636363636 +/- 0.015645610373741962
-191.8867915368117 +/- 12.006234135702153
0.9120418743646127 +/- 0.016610607242537902
0.7726257669199244 +/- 0.025511456456983394


#### nl = 100, nu = 550

In [239]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 100,nu = 550, cls = PNB, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8426363636363637 +/- 0.020678371341145658
-282.36705726097426 +/- 22.11292538614638
0.7734199673241845 +/- 0.045786784428532096
0.6699732196782135 +/- 0.036145452240484766


In [240]:
# PTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 100,nu = 550, cls = PTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9101818181818182 +/- 0.014561354945582488
-193.22422184394105 +/- 16.568674476254866
0.8808754905150717 +/- 0.0315241362938831
0.8220341811215256 +/- 0.028756876858128125


In [241]:
# PSTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 100,nu = 550, cls = PSTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.9056363636363637 +/- 0.013210314027085623
-198.3752925275218 +/- 15.781372369165025
0.8638833472307578 +/- 0.027976124626683446
0.8269364209824677 +/- 0.026160925353804033


In [242]:
# PESTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 100,nu = 550, cls = PESTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.899818181818182 +/- 0.012673914122067965
-199.59239036696513 +/- 14.127826711100976
0.8492596689205396 +/- 0.026105795667302138
0.8268012950434874 +/- 0.02352833510856779


In [244]:
# WNB case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 100,nu = 550, cls = WNB, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8318181818181818 +/- 0.019628786416850813
-283.8535073790839 +/- 18.915798902467802
0.7665782908970042 +/- 0.05041729488932575
0.621312300882637 +/- 0.030646769171525503


In [245]:
# WTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 100,nu = 550, cls = WTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8863636363636365 +/- 0.015086527293752006
-217.35164292177586 +/- 14.199136222545615
0.8747544399218986 +/- 0.022960221137181453
0.7444730143204581 +/- 0.03141139808294125


#### nl = 50, nu = 550

In [246]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 50,nu = 550, cls = PNB, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8332727272727272 +/- 0.02053096834435786
-292.55328726908414 +/- 27.72637892774799
0.7489865007407153 +/- 0.03854345608869164
0.6543756882497338 +/- 0.03601288437520223


In [247]:
# PTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 50,nu = 550, cls = PTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8876363636363637 +/- 0.01839511253809709
-224.14320445912898 +/- 18.984993842948846
0.8468622603765826 +/- 0.04205704482402858
0.7746106232193928 +/- 0.03506230127584673


In [248]:
# PSTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 50,nu = 550, cls = PSTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8846363636363636 +/- 0.01800344319684845
-223.5178742690561 +/- 20.928249967755953
0.8351371980052089 +/- 0.036605870318342074
0.7775983538507216 +/- 0.035067298619388114


In [249]:
# PESTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 50,nu = 550, cls = PESTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8887272727272725 +/- 0.015207653575510167
-217.47965443668392 +/- 19.15708068063602
0.8371926350850536 +/- 0.02752019577005378
0.7940009873670496 +/- 0.03687519021286066


In [250]:
# WNB case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 50,nu = 550, cls = WNB, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8282727272727272 +/- 0.019177078259696192
-291.05897781235774 +/- 20.39272039189562
0.7588981501669465 +/- 0.036396433699721974
0.6112050442087475 +/- 0.03320532454787907


In [251]:
# WTAN case control
Accuracy, CLL, Precision,Recall = get_cv(X2,y2,case_control = True,nl = 50,nu = 550, cls = WTAN, M =M2, verbose = False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8658181818181818 +/- 0.015752213539847626
-240.33256410949258 +/- 15.982768560196314
0.8406713756601443 +/- 0.02826529005118681
0.7012126804666987 +/- 0.030164065577275995


## P450

### single training

#### c = 0.7, nt = 1000

In [178]:
# PNB single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PNB, M =M,runs= 20,c = 0.7,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7111485749657473 +/- 0.022962218767732478
-449.1469100293324 +/- 25.712488902857356
0.741227188831835 +/- 0.016775995610709353
0.7550288000500099 +/- 0.018673716177036982


In [179]:
# PTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PTAN, M =M,runs= 20,c = 0.7,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7622183629528766 +/- 0.029105260710230094
-472.32297108747764 +/- 58.26347208735815
0.7749400472387886 +/- 0.022511786803459985
0.7968954193193457 +/- 0.021795549286826553


In [180]:
# PSTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PSTAN, M =M,runs= 20,c = 0.7,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7643458319976613 +/- 0.02138417279527099
-459.5755293593869 +/- 41.92736642916907
0.7754668198655184 +/- 0.016867161528774974
0.798126653933289 +/- 0.017384409353976848


In [181]:
# PESTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PESTAN, M =M,runs= 20,c = 0.7,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7595599565975293 +/- 0.02009469844453975
-441.4635513023236 +/- 43.15036747858927
0.7728663362935446 +/- 0.01618185539247302
0.794804093236894 +/- 0.0169577206328536


In [182]:
# WNB single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = WNB, M =M,runs= 20,c = 0.7,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7203733816467631 +/- 0.023757829660318943
-436.3748980487691 +/- 21.85253125392594
0.7449027273410886 +/- 0.01558278525180776
0.7608355919705907 +/- 0.017935027343797722


In [183]:
# WTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = WTAN, M =M,runs= 20,c = 0.7,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7992389713133893 +/- 0.023258891816601072
-362.55313087844945 +/- 19.777818369544036
0.7956755825397429 +/- 0.018077100922502833
0.8201991246396041 +/- 0.018075508703300706


#### c = 0.5, nt = 1000

In [184]:
# PNB single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PNB, M =M,runs= 20,c = 0.5,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7391914795053851 +/- 0.02180823210479563
-518.9729406535849 +/- 30.229723473814186
0.7667207681531598 +/- 0.019829806038464685
0.7448676917755066 +/- 0.020730561165982677


In [185]:
# PTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PTAN, M =M,runs= 20,c = 0.5,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7706139166319543 +/- 0.03239151925425723
-570.0792351405473 +/- 82.84375004877917
0.792348106481405 +/- 0.026400312285831642
0.7757218921093689 +/- 0.029654417837569856


In [186]:
# PSTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PSTAN, M =M,runs= 20,c = 0.5,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7747495550396628 +/- 0.01706731629543579
-536.5374355073683 +/- 46.54816249298304
0.7957174654457376 +/- 0.016038099249110063
0.7797104584119477 +/- 0.016914622518453076


In [187]:
# PESTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PESTAN, M =M,runs= 20,c = 0.5,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7742945885525079 +/- 0.018045787658957
-516.3431190782094 +/- 41.07894674359527
0.796975847086318 +/- 0.015889621350414596
0.779379580444236 +/- 0.01791161148504767


In [188]:
# WNB single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = WNB, M =M,runs= 20,c = 0.5,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7462421838666906 +/- 0.022933347693629723
-514.3477030564594 +/- 21.191143729654627
0.7697321589490217 +/- 0.02046191640225928
0.7513426242055621 +/- 0.021486330882494555


In [189]:
# WTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = WTAN, M =M,runs= 20,c = 0.5,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8085010581758455 +/- 0.02037566943253297
-449.37698138844826 +/- 19.537904769067364
0.8164583554058658 +/- 0.017909091928955647
0.8112331808404933 +/- 0.019752203928938512


#### c = 0.3, nt = 1000

In [190]:
# PNB single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PNB, M =M,runs= 20,c = 0.3,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7477847672700197 +/- 0.02160439213841777
-609.8760777909421 +/- 34.829148301923325
0.7601053397116919 +/- 0.02250080285345082
0.7275674893937918 +/- 0.0220108036306155


In [191]:
# PTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PTAN, M =M,runs= 20,c = 0.3,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7500067623597746 +/- 0.02844013390647548
-750.0400033180783 +/- 85.28568489240233
0.7606586735008666 +/- 0.03182674654476227
0.73130168869892 +/- 0.029015129202409215


In [192]:
# PSTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PSTAN, M =M,runs= 20,c = 0.3,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.765801094311992 +/- 0.030019877197048593
-695.0854190990691 +/- 94.16607712555968
0.7769320405574065 +/- 0.03418108668631804
0.747902242798583 +/- 0.032139962568321044


In [193]:
# PESTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PESTAN, M =M,runs= 20,c = 0.3,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7729461275378263 +/- 0.025127177377813715
-648.7735776765024 +/- 68.28331011512914
0.7860125957271344 +/- 0.02640998511393842
0.7544161105894185 +/- 0.027881854755380023


In [194]:
# WNB single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = WNB, M =M,runs= 20,c = 0.3,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7594370647683498 +/- 0.017361350762962684
-606.7515349699951 +/- 20.950548327488697
0.7685766823040024 +/- 0.018285777095220987
0.7416620953503934 +/- 0.018481650263056523


In [195]:
# WTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = WTAN, M =M,runs= 20,c = 0.3,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7939830408662235 +/- 0.021242090517254217
-555.729061531296 +/- 23.32745400144292
0.7952843222995314 +/- 0.021745576689418553
0.7838186268284184 +/- 0.022272556135449404


#### c = 0.2, nt = 1000

In [196]:
# PNB single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PNB, M =M,runs= 20,c = 0.2,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7454165470832341 +/- 0.02064800995591434
-688.6525799842746 +/- 51.402752928172056
0.7443079352465434 +/- 0.023339333007330813
0.716552875515067 +/- 0.021974874429242192


In [197]:
# PTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PTAN, M =M,runs= 20,c = 0.2,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7276334013486702 +/- 0.031753279980326274
-922.4930718761896 +/- 116.34003698236894
0.7242406536834798 +/- 0.03473132210852176
0.6974831491424606 +/- 0.03366887486814032


In [198]:
# PSTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PSTAN, M =M,runs= 20,c = 0.2,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7459869684349308 +/- 0.01884113851790203
-815.4086670645614 +/- 103.29558295925477
0.7439567758153324 +/- 0.020003391142247893
0.718643532273807 +/- 0.02146726938537791


In [199]:
# PESTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PESTAN, M =M,runs= 20,c = 0.2,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7528750186829807 +/- 0.024384379097411937
-744.6256049283231 +/- 79.49184163024518
0.7544289712798743 +/- 0.028844781480277498
0.7230619543181677 +/- 0.026016399032966805


In [200]:
# WNB single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = WNB, M =M,runs= 20,c = 0.2,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7513908185062139 +/- 0.019253779087104017
-661.5271461621421 +/- 18.664529747666716
0.7478507546274108 +/- 0.022032062232026355
0.726513586299047 +/- 0.020868161205636542


In [201]:
# WTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = WTAN, M =M,runs= 20,c = 0.2,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7812325535306738 +/- 0.020087969415753873
-618.9517114990605 +/- 22.418315144659243
0.775382273420987 +/- 0.023486413840290175
0.7655956295809484 +/- 0.0213849562632329


#### c = 0.1, nt = 1000

In [204]:
# PNB single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PNB, M =M,runs= 20,c = 0.1,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7126237577725638 +/- 0.028622406339543893
-856.0991767484305 +/- 81.19122865682529
0.6934819309413436 +/- 0.035071893095399265
0.6742482425298707 +/- 0.03231354790404217


In [205]:
# PTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PTAN, M =M,runs= 20,c = 0.1,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.687194927515349 +/- 0.027864967042543022
-1313.0166518556314 +/- 184.31711191872148
0.6638697275245942 +/- 0.03309226780071941
0.6457047210754364 +/- 0.029082282232052257


In [206]:
# PSTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PSTAN, M =M,runs= 20,c = 0.1,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7146104560355346 +/- 0.0246873687027064
-1080.6729873470713 +/- 152.25054139099987
0.6965047799704693 +/- 0.03057043379940947
0.6745432233753693 +/- 0.029217071606602044


In [207]:
# PESTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = PESTAN, M =M,runs= 20,c = 0.1,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7189291133200915 +/- 0.027492779222033793
-997.8533039099209 +/- 104.37957496683288
0.7036126671748424 +/- 0.03799728979877876
0.6771311877412636 +/- 0.030036508046891352


In [210]:
# WNB single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = WNB, M =M,runs= 20,c = 0.1,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7204228517708462 +/- 0.02135742797549042
-746.4779170865115 +/- 28.900826682688333
0.7014086263226302 +/- 0.026036169222912107
0.6929270770944892 +/- 0.026115643615634013


In [209]:
# WTAN single training
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = False, cls = WTAN, M =M,runs= 20,c = 0.1,n_T =1000, verbose=False)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7429749935080089 +/- 0.023289604728684344
-716.5872115791071 +/- 28.03097285984815
0.7260487165910441 +/- 0.027275010306874955
0.7222558183110327 +/- 0.028595401615161314


### case-control

#### nl = 600, nu = 1000

In [143]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 600,nu = 1000, cls = PNB, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.79505 +/- 0.022399720980405085
-665.0382368543089 +/- 42.24681315111548
0.7872894333498944 +/- 0.026296041239938485
0.7489640523341479 +/- 0.024754302041094748


In [144]:
# PTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 600,nu = 1000, cls = PTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.79365 +/- 0.025720176904523797
-760.5003316958503 +/- 109.30091414730643
0.784666558289416 +/- 0.027089891552064906
0.7485081383480338 +/- 0.030345357348508052


In [145]:
# PSTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 600,nu = 1000, cls = PSTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8166 +/- 0.014492756811593822
-689.9123585298273 +/- 71.77730924240512
0.8117007189730359 +/- 0.014787609534625724
0.7757266289513362 +/- 0.018496238876419522


In [146]:
# PESTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 600,nu = 1000, cls = PESTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8165999999999999 +/- 0.017471119025408734
-658.403112197849 +/- 62.3039353400051
0.8137364222641139 +/- 0.016737100671805418
0.7731945114394678 +/- 0.02186961966677162


In [147]:
# WNB # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 600,nu = 1000, cls = WNB, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7881 +/- 0.017314444836609707
-717.9090804296668 +/- 23.238802703889824
0.7788505738121901 +/- 0.019188144923241054
0.740674323718425 +/- 0.020885209626206564


In [148]:
# WTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 600,nu = 1000, cls = WTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8307499999999999 +/- 0.018082795690932296
-652.2697677165869 +/- 22.375631333116104
0.8167038756250466 +/- 0.020224923424727113
0.8078592637700182 +/- 0.020351120744103276


#### nl = 400, nu = 1000

In [149]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 400,nu = 1000, cls = PNB, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7891999999999999 +/- 0.022907640646736185
-675.8582935427686 +/- 46.80458704677498
0.7805992981537061 +/- 0.027321405510037557
0.7417171328296371 +/- 0.024771083588357553


In [150]:
# PTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 400,nu = 1000, cls = PTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7855999999999999 +/- 0.021802293457340662
-799.4324766050911 +/- 94.37690696971342
0.7743097818143372 +/- 0.022712193906806028
0.7400127151464416 +/- 0.02485978131656679


In [151]:
# PSTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 400,nu = 1000, cls = PSTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8041499999999999 +/- 0.018371921510827307
-736.0646566377693 +/- 80.78058131179934
0.7980227879741748 +/- 0.01910289730315773
0.7601225856404229 +/- 0.02200831976116297


In [152]:
# PESTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 400,nu = 1000, cls = PESTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.80615 +/- 0.022634652637051875
-696.1798184461585 +/- 64.41218080412027
0.8025921183215645 +/- 0.02400918815294093
0.7597997125421662 +/- 0.02603844796092751


In [153]:
# WNB # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 400,nu = 1000, cls = WNB, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7854000000000001 +/- 0.020664946164943184
-719.9708455022039 +/- 23.233251884505954
0.773327349079201 +/- 0.024514442438936634
0.7409812804361273 +/- 0.02239828431203


In [154]:
# WTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 400,nu = 1000, cls = WTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.82205 +/- 0.020394791001625855
-663.6065035816862 +/- 22.31788707670194
0.8071365129783608 +/- 0.022893212592704867
0.7978570202411658 +/- 0.0232741825648246


#### nl = 300, nu = 1000

In [155]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 300,nu = 1000, cls = PNB, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7824000000000001 +/- 0.018679935760060863
-700.4212815478403 +/- 54.43251536489727
0.7731991597271064 +/- 0.02275464442931776
0.7330516522366731 +/- 0.02047514249918806


In [156]:
# PTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 300,nu = 1000, cls = PTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.77595 +/- 0.024270300780995704
-821.4934661035575 +/- 103.85245344492478
0.7616921587090404 +/- 0.028561725934159867
0.7309667787932053 +/- 0.025338657561298397


In [157]:
# PSTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 300,nu = 1000, cls = PSTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7934500000000001 +/- 0.012326698665904011
-767.2710532234335 +/- 65.87269881543665
0.783862668869134 +/- 0.015432725782194161
0.7489271116840412 +/- 0.016631857589209532


In [158]:
# PESTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 300,nu = 1000, cls = PESTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.80015 +/- 0.01470807601285768
-720.2939658746674 +/- 52.27622056525653
0.7947112485531923 +/- 0.018336642614940148
0.7537772041585126 +/- 0.018065910219524403


In [159]:
# WNB # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 300,nu = 1000, cls = WNB, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7809000000000001 +/- 0.01937240305176414
-724.8922110170239 +/- 22.02663916589762
0.76815406733575 +/- 0.023246607073911473
0.736350494807834 +/- 0.020200721442747708


In [160]:
# WTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 300,nu = 1000, cls = WTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8131 +/- 0.02409543525234601
-674.151441318528 +/- 23.945071698588492
0.7974584088222183 +/- 0.026500738586199078
0.7873238186556993 +/- 0.02832647309223


#### nl = 200, nu = 1000

In [161]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 200,nu = 1000, cls = PNB, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7725500000000001 +/- 0.019314437605066338
-728.7485970191094 +/- 65.2066190931942
0.7588020681455966 +/- 0.023044815533970878
0.7243548493907253 +/- 0.02143904684626869


In [162]:
# PTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 200,nu = 1000, cls = PTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7523 +/- 0.02523905703468339
-947.5772090822909 +/- 117.52116686636022
0.7325028995371938 +/- 0.03324831563992487
0.7049565598913949 +/- 0.024483739314595115


In [164]:
# PSTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 200,nu = 1000, cls = PSTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7771999999999999 +/- 0.017993332098307977
-835.5767334382259 +/- 64.49353831550545
0.7634205192448059 +/- 0.023866380114199007
0.7307429308400798 +/- 0.023359109264613858


In [165]:
# PESTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 200,nu = 1000, cls = PESTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7841 +/- 0.01898394058144937
-773.6729832262897 +/- 68.48526434544095
0.7742883950264042 +/- 0.02612289786378475
0.7356451276483384 +/- 0.023832007839931862


In [166]:
# WNB # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 200,nu = 1000, cls = WNB, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7721 +/- 0.022543069888548914
-731.5970476872856 +/- 25.27897610688403
0.7554098381444014 +/- 0.02632237083739065
0.7307626911146567 +/- 0.02462459668190787


In [167]:
# WTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 200,nu = 1000, cls = WTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.8008500000000002 +/- 0.02195056946869486
-691.1864236285953 +/- 26.37179127653948
0.784002395531812 +/- 0.02448614213722023
0.7734261133511977 +/- 0.027010440171751342


#### nl = 100, nu = 1000

In [168]:
# PNB case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 100,nu = 1000, cls = PNB, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.74095 +/- 0.04122071687877348
-835.0690225400944 +/- 127.28221749783714
0.7187834226762098 +/- 0.05427015369283586
0.6919585114224568 +/- 0.044126845182638194


In [169]:
# PTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 100,nu = 1000, cls = PTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.6985 +/- 0.029914043524739337
-1259.3530699981493 +/- 162.11061085141358
0.6661743261736855 +/- 0.0404389583754027
0.6456375141682107 +/- 0.033403350677355434


In [170]:
# PSTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 100,nu = 1000, cls = PSTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7355500000000001 +/- 0.03271005197183277
-1044.2938386394999 +/- 134.11652020851142
0.7117425480583716 +/- 0.04101372681728621
0.68587217332498 +/- 0.034209310055114


In [171]:
# PESTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 100,nu = 1000, cls = PESTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.74285 +/- 0.023365091482808284
-954.4824280115733 +/- 86.68758904526379
0.7222113896301809 +/- 0.032598484900630505
0.6912845942384246 +/- 0.024240679908238586


In [172]:
# WNB # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 100,nu = 1000, cls = WNB, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7483000000000002 +/- 0.033914746055366554
-767.0236149895861 +/- 42.5439172248315
0.7245515726541134 +/- 0.04262103916548036
0.7119009433920997 +/- 0.03768815015523453


In [173]:
# WTAN # case control
Accuracy, CLL, Precision,Recall = get_cv(X,y,case_control = True,nl = 100,nu = 1000, cls = WTAN, M =M, verbose=False,runs= 20)
print("%s +/- %s" % (np.mean(Accuracy), np.std(Accuracy) ) )
print("%s +/- %s" % (np.mean(CLL), np.std(CLL) ) )
print("%s +/- %s" % (np.mean(Precision), np.std(Precision) ) )
print("%s +/- %s" % (np.mean(Recall), np.std(Recall) ) )

0.7679 +/- 0.035424426600864
-739.7905180119633 +/- 42.40120610401264
0.7456591637248926 +/- 0.042021783740960175
0.7428628686412168 +/- 0.04058211834429943
